# 1. Pointcloud Visualization

##### Idea

To visualize the given LIDAR pointcloud data we first needed to convert the latitude and longitude information to a cartesian coordinate system that has X, Y and Z values to specify a single point. We can do this by using the latitude and longitude information to calculate the UTM coordinates for each point. As a result we will get a value for the easting (X) and northing (Y). The altitude value, which is already given as meters can thus be seen as the Z value in our new coordinate system. After saving the data to a new file we can then use http://lidarview.com/ to visualize the pointcloud to get a first impression of the data.

##### Problems

Our initial thought was that the intesity of each point in our point cloud is very important to determine the the lane markings in the point cloud. In fact, the lane marking are on a flat surface and the marking itself can also be seen as a flat line on the flat surface. Therfore, we cannot use the shape or variance in the point cloud to detect the lane markings. However, the intensity of each LIDAR point should contain enough information to recover the lane marking from the flat surface since the lane marking should have a higher intensity compared to the plain asphalt or concrete of the street. The problem during the visualization was that the visualization tool that we were using could not visualize the intensity of each point. As you can see in the following image, the visualization uses different colors to denote the elevation instead of the intensity of each point. Although the tool had an option for choosing the intensity the visualization did not work, maybe because the intensity was in a wrong format.

##### Results
![Full LIDAR Pointcloud Visualization](FullView.png "Full LIDAR Pointcloud View")

###### Code

In [1]:
import numpy as np
import pandas as pd

import utm

In [2]:
data = []

with open('./final_project_point_cloud.fuse') as f:  
    line = f.readline()
    while line:
        d = line.split()
        data.append(d)
        line = f.readline()
        
a = np.array(data)

In [3]:
pointcloud_df = pd.DataFrame()

pointcloud_df["Latitude"] = a[:,0]
pointcloud_df["Longitude"] = a[:,1]
pointcloud_df["Altitude"] = a[:,2]
pointcloud_df["Intensity"] = a[:,3]
pointcloud_df["Latitude"] = pd.to_numeric(pointcloud_df["Latitude"])
pointcloud_df["Longitude"] = pd.to_numeric(pointcloud_df["Longitude"])
pointcloud_df["Altitude"] = pd.to_numeric(pointcloud_df["Altitude"])
pointcloud_df["Intensity"] = pd.to_numeric(pointcloud_df["Intensity"])
pointcloud_df["Easting"] = pointcloud_df.apply(lambda x: utm.from_latlon(x["Latitude"], x["Longitude"])[0], axis = 1)
pointcloud_df["Northing"] = pointcloud_df.apply(lambda x: utm.from_latlon(x["Latitude"], x["Longitude"])[1], axis = 1)
pointcloud_df["Easting"] = pointcloud_df["Easting"] - pointcloud_df["Easting"].min()
pointcloud_df["Northing"] = pointcloud_df["Northing"] - pointcloud_df["Northing"].min()
pointcloud_df["Altitude"] = pointcloud_df["Altitude"] - pointcloud_df["Altitude"].min()

pointcloud_df.head()

,Latitude,Longitude,Altitude,Intensity,Easting,Northing
0,45.903883,11.028414,10.6604,10,108.226321,65.043547
1,45.903683,11.028221,12.6662,5,93.822923,42.445590
2,45.903683,11.028222,12.6415,7,93.938542,42.355143
3,45.903682,11.028224,12.6263,7,94.080686,42.289830
4,45.903681,11.028226,12.6188,7,94.233803,42.220349


In [4]:
xyzi_df = pointcloud_df[["Easting", "Northing", "Altitude", "Intensity"]]
xyzi_df.to_csv("pointcloud.fuse", sep=" ", header=False, index=False)
xyzi_df.head()

,Easting,Northing,Altitude,Intensity
0,108.226321,65.043547,10.6604,10
1,93.822923,42.445590,12.6662,5
2,93.938542,42.355143,12.6415,7
3,94.080686,42.289830,12.6263,7
4,94.233803,42.220349,12.6188,7


# 2. Lane Marking Detection

##### Idea
- Maybe set a threshold and then try to identify lines within the thresholds

##### Problems
- We do not have a lot of data so we cannot use a deep learning algorithm to identify lanes

##### Results
- Show an image of our final lane marking